In [2]:
import numpy as np
import numpy.typing as npt
from scipy.ndimage import binary_closing, binary_opening
from skimage.filters import gaussian, hessian, threshold_li
from skimage.measure import label, regionprops
from skimage.morphology import disk, remove_small_holes


@staticmethod
def binarize(image: npt.NDArray, threshold: float = 0.1):
    """
    Binarize an image based on a threshold.

    Parameters
    ----------
    image : npt.NDArray
        Numpy array of the image to binarize.
    threshold : float
        Threshold value to binarize the image. Default is 0.1.

    Returns
    -------
    npt.NDArray
        Binarized image as a Numpy array.
    """
    cleaned = np.where(image <= threshold, 0, 1)
    return cleaned.astype(np.uint8)


@staticmethod
def safe_opening(
    image: np.ndarray,
    radius: float = 1.0,
    safe_area_threshold: float = 2.0,
    max_ratio_of_width_to_length: float = 1.0,
) -> np.ndarray:
    """
    Apply a safe opening operation to the image.

    Parameters
    ----------
    image : npt.NDArray
        Numpy array of the image to process.
    radius : float
        Radius for the opening operation in pixels.
    safe_area_threshold : float
        Threshold for the area of the objects to be removed in nanometers squared.
    max_ratio_of_width_to_length : float
        Maximum ratio of width to length for the objects to be removed.

    Returns
    -------
    npt.NDArray
        Numpy array of the processed image.
    """
    pixel_to_nm_scaling = 1.0  # Assuming a default scaling factor, can be adjusted as needed
    # Apply binary opening
    opened = binary_opening(image, disk(radius)).astype(np.uint8)
    removed = image - opened
    labels = label(removed)
    # Maintain large objects that were opened
    props = regionprops(labels)
    for i, prop in enumerate(props):
        if prop.major_axis_length == 0:
            continue
        if (
            prop.area > safe_area_threshold / (pixel_to_nm_scaling**2)
            or (prop.area / (prop.major_axis_length**2)) > max_ratio_of_width_to_length
        ):
            # Remove the large object from the removal mask so it will remain in the image
            removed[labels == i + 1] = 0
    return image - removed


@staticmethod
def apply_opening(
    image: np.ndarray,
    pixel_to_nm_scaling: float = 1.0,
    radius: float = 1.0,
    use_safe_opening: bool = True,
    safe_area_threshold: float = 2.0,
    max_ratio_of_width_to_length: float = 1.0,
):
    """
    Apply an opening operation to the image.

    Parameters
    ----------
    image : np.ndarray
        Numpy array of the image to process.
    pixel_to_nm_scaling : float
        Scaling of pixels to nanometres.
    radius : float
        Radius for the opening operation in nanometers.
    use_safe_opening : bool
        Whether to use the safe opening method or the standard binary opening.
    safe_area_threshold : float
        Threshold for the area of the objects to be removed in nanometers squared.
    max_ratio_of_width_to_length : float
        Maximum ratio of width to length for the objects to be removed.

    Returns
    -------
    np.ndarray
        Numpy array of the processed image after opening.
    """
    if use_safe_opening:
        image = safe_opening(
            image,
            radius=np.floor(radius / pixel_to_nm_scaling),
            safe_area_threshold=safe_area_threshold,
            max_ratio_of_width_to_length=max_ratio_of_width_to_length,
        )
    else:
        image = binary_opening(image, disk(np.floor(radius / pixel_to_nm_scaling))).astype(np.uint8)
    return image


@staticmethod
def split_ridges(
    image: npt.NDArray[np.float64],
    pixel_to_nm_scaling: float = 0.34,
    open_at_start: bool = True,
    closing_iterations_at_end: int = 1,
    opening_iterations_at_end: int = 1,
    small_holes_threshold: int = 50,
    hessian_sigmas_nm: list[int] | None = None,
    gaussian_blurring_sigma: float = 0.0,
    opening_radius: float = 2.0,
    closing_radius: float = 2.0,
    use_safe_opening: bool = True,
    safe_area_threshold: float = 2.0,
    max_ratio_of_width_to_length: float = 1.0,
) -> npt.NDArray[np.bool_]:
    """
    Split ridges in the image using Hessian matrix and binary opening.

    Parameters
    ----------
    image : npt.NDArray[np.float64]
        Numpy array of the image to process.
    pixel_to_nm_scaling : float
        Scaling of pixels to nanometres.
    open_at_start : bool
        Whether to apply opening to the Hessian result at the start.
    closing_iterations_at_end : int
        Number of closing iterations to apply at the end.
    opening_iterations_at_end : int
        Number of opening iterations to apply at the end.
    small_holes_threshold : int | None
        Threshold for removing small holes in nanometers. If None, no holes are removed.
    hessian_sigmas_nm : list[int] | None
        List of sigmas in nanometers for the Hessian matrix. If None, default values
    gaussian_blurring_sigma : float
        Sigma for Gaussian blurring to smooth the result in nm.
    opening_radius : float
        Radius for the opening operation in nanometers.
    closing_radius : float
        Radius for the closing operation in nanometers.
    use_safe_opening : bool
        Whether to use the safe opening method or the standard binary opening.
    safe_area_threshold : float
        Threshold for the area of the objects to be removed in nanometers squared.
    max_ratio_of_width_to_length : float
        Maximum ratio of width to length for the objects to be removed.

    Returns
    -------
    npt.NDArray
        Numpy array of the full mask tensor with ridges split. The tensor is made of two 2D arrays, one being the
        identified shapes and the other being the background (the reverse of that).
    """
    if hessian_sigmas_nm is None:
        hessian_sigmas_nm = [1, 2, 3]
    # Use Li thresholding to create a binary mask of the image
    threshold = threshold_li(image)
    threshold_li_result = binarize(image > threshold)

    # Compute the Hessian matrix to split ridges
    sigmas = [nm / pixel_to_nm_scaling for nm in hessian_sigmas_nm]
    hessian_result = hessian(image, black_ridges=True, sigmas=sigmas)

    # Clean the Hessian result by thresholding to a binary image
    cleaned_hessian = binarize(hessian_result)

    # Apply opening to the Hessian result if specified
    if open_at_start:
        final_hessian = apply_opening(
            cleaned_hessian,
            pixel_to_nm_scaling=pixel_to_nm_scaling,
            radius=opening_radius,
            use_safe_opening=use_safe_opening,
            safe_area_threshold=safe_area_threshold,
            max_ratio_of_width_to_length=max_ratio_of_width_to_length,
        )
    else:
        final_hessian = cleaned_hessian

    # Remove black borders (by filling in white background) by applying a mask
    ridges_split = final_hessian * threshold_li_result

    # Remove small holes if specified. This is so that the binary opening does not create small holes
    if small_holes_threshold > 0:
        ridges_split = remove_small_holes(
            ridges_split.astype(bool), area_threshold=np.floor(small_holes_threshold / pixel_to_nm_scaling)
        ).astype(np.uint8)

    # Perform closing and opening alternatively to smooth the result the number of times specified
    while closing_iterations_at_end > 0 and opening_iterations_at_end > 0:
        ridges_split = apply_opening(
            image=ridges_split,
            pixel_to_nm_scaling=pixel_to_nm_scaling,
            radius=opening_radius,
            use_safe_opening=use_safe_opening,
            safe_area_threshold=safe_area_threshold,
            max_ratio_of_width_to_length=max_ratio_of_width_to_length,
        )
        opening_iterations_at_end -= 1
        ridges_split = binary_closing(ridges_split, disk(np.floor(closing_radius / pixel_to_nm_scaling)))
        closing_iterations_at_end -= 1

    # If there are still iterations left (if more opening needs to be done than closing or vice versa), apply them at the end
    while opening_iterations_at_end > 0:
        ridges_split = apply_opening(
            image=ridges_split,
            pixel_to_nm_scaling=pixel_to_nm_scaling,
            radius=opening_radius,
            use_safe_opening=use_safe_opening,
            safe_area_threshold=safe_area_threshold,
            max_ratio_of_width_to_length=max_ratio_of_width_to_length,
        )
        opening_iterations_at_end -= 1
    while closing_iterations_at_end > 0:
        ridges_split = binary_closing(ridges_split, disk(np.floor(closing_radius / pixel_to_nm_scaling)))
        closing_iterations_at_end -= 1

    # Again remove small holes in the mask tensor if specified. These can sometimes be created from the binary opening
    if small_holes_threshold > 0:
        ridges_split = remove_small_holes(
            ridges_split.astype(bool),
            area_threshold=np.floor(small_holes_threshold / (pixel_to_nm_scaling**2)),
        ).astype(np.uint8)

    # Apply Gaussian blurring to further smooth the result
    if gaussian_blurring_sigma > 0:
        ridges_split = gaussian(ridges_split, sigma=gaussian_blurring_sigma / pixel_to_nm_scaling)

    return np.stack(arrays=(1 - ridges_split, ridges_split), axis=-1)

image = np.load("resources/ridges_test_images/test_image_1.npy")
result = split_ridges(image)
np.save("resources/ridges_test_images/expected_image_1.npy", result)
